In [1]:
from stox.imports import *
from stox.api import IB_Extractor
from stox import misc, yahoo
from omnibelt import tqdmd_notebook as tqdmd
from omnibelt import save_yaml
from collections import Counter
# from stox.alpha_vantage import AlphaVantageStocks

yahoo_root = misc.yahoo_root()
yahoo_root = yahoo_root.parent.parent / 'testdata' / 'yahoo_data'
print(yahoo_root)

C:\Users\anwan\OneDrive\Khan\projects\stox\testdata\yahoo_data


In [153]:
matches = {}

currency = 'EUR'
currency = 'GBP'

In [154]:
# cts = ibe.search('Dr. Ing. h.c. F. Porsche')
def get_matches(query):
    cts = ibe.search(query)
    matches = list(misc.best_matches([c for c in cts if c.secType == 'STK' and c.currency == currency], [query],
                                     key1=lambda c: c.description))
    return matches

In [172]:
root = misc.assets_root()
path = root / 'symbols.yml'
data = load_export(path)
existing = Counter(row['yahoo'] for row in data)
len(data), len(existing), existing.most_common(2)

(271, 271, [('AAPL', 1), ('MC.PA', 1)])

In [144]:
raw = '''RR.L	Rolls-Royce Holdings plc
ENT.L	Entain Plc
RS1.L	RS Group plc
SPX.L	Spirax-Sarco Engineering plc
AHT.L	Ashtead Group plc
CCH.L	Coca-Cola HBC AG
RTO.L	Rentokil Initial plc
BATS.L	British American Tobacco p.l.c.
ANTO.L	Antofagasta plc
BA.L	BAE Systems plc
VOD.L	Vodafone Group Public Limited Company
SMIN.L	Smiths Group plc
SDR.L	Schroders plc
HLN.L	Haleon plc
CNA.L	Centrica plc
PSN.L	Persimmon Plc
TSCO.L	Tesco PLC
EXPN.L	Experian plc
SHEL.L	Shell plc
CPG.L	Compass Group PLC
STJ.L	St. James's Place plc
AUTO.L	Auto Trader Group plc
SSE.L	SSE plc
SMT.L	Scottish Mortgage Investment Trust PLC
ABDN.L	abrdn plc
MNG.L	M&G plc
RMV.L	Rightmove plc
FRAS.L	Frasers Group plc
AAF.L	Airtel Africa Plc
PRU.L	Prudential plc'''.split('\n')
companies = [row.split('\t') for row in raw]
companies = [(tk, name.lower().replace('.br','').replace('.', '').replace(' spa', '').replace(' nv', '').replace(' sa', '').replace(' plc', '').replace(' aktiengesellschaft', '').replace(' ag', '').replace(' se', '')) for tk, name in companies]
# companies = [(f'{tk}.HE', name) for tk, name in companies]
companies

[('RR.L', 'rolls-royce holdings'),
 ('ENT.L', 'entain'),
 ('RS1.L', 'rs group'),
 ('SPX.L', 'spirax-sarco engineering'),
 ('AHT.L', 'ashtead group'),
 ('CCH.L', 'coca-cola hbc'),
 ('RTO.L', 'rentokil initial'),
 ('BATS.L', 'british american tobacco'),
 ('ANTO.L', 'antofagasta'),
 ('BA.L', 'bae systems'),
 ('VOD.L', 'vodafone group public limited company'),
 ('SMIN.L', 'smiths group'),
 ('SDR.L', 'schroders'),
 ('HLN.L', 'haleon'),
 ('CNA.L', 'centrica'),
 ('PSN.L', 'persimmon'),
 ('TSCO.L', 'tesco'),
 ('EXPN.L', 'experian'),
 ('SHEL.L', 'shell'),
 ('CPG.L', 'compass group'),
 ('STJ.L', "st james's place"),
 ('AUTO.L', 'auto trader group'),
 ('SSE.L', 'sse'),
 ('SMT.L', 'scottish mortgage investment trust'),
 ('ABDN.L', 'abrdn'),
 ('MNG.L', 'm&g'),
 ('RMV.L', 'rightmove'),
 ('FRAS.L', 'frasers group'),
 ('AAF.L', 'airtel africa'),
 ('PRU.L', 'prudential')]

In [145]:
bad_yk = [tk for tk, _ in tqdmd(companies, key=lambda x: x[0]) if len(yf.Ticker(tk).info) == 0]
bad_yk

  0%|          | 0/30 [00:00<?, ?it/s]

[]

In [146]:
todo = [c[1] for c in companies]

In [56]:
ibe = IB_Extractor(True)

ERROR:ib_insync.wrapper: Error 321, reqId 2147483647: Error validating request.-'cG' : cause - The API interface is currently in Read-Only mode.
ERROR:ib_insync.wrapper: Error 321, reqId -1: Error validating request.-'ce' : cause - The API interface is currently in Read-Only mode.
ERROR:ib_insync.ib: open orders request timed out
ERROR:ib_insync.ib: completed orders request timed out


In [155]:
for query in tqdmd(todo, key=True):
    if not ibe.ib.isConnected():
        ibe = IB_Extractor(True)
    if query not in matches:
        try:
            matches[query] = get_matches(query)
        except ConnectionError:
            matches[query] = []

  0%|          | 0/30 [00:00<?, ?it/s]

In [156]:
tbl = [(query, tk, [m[1].symbol for m in matches.get(query, [])]) for tk, query in companies]
print(tabulate(tbl))

-------------------------------------  ------  -------------------------------------------------------------------------------------------------------------
rolls-royce holdings                   RR.L    ['RR.', 'RR.CSHS', 'RRN', 'RRF.GBP', 'RR.C.SPO', 'RR.CONV', 'RR.C.CSH', 'RR.C.STK', 'RR.C.CASH', 'RR.C.STOC']
entain                                 ENT.L   ['ENT']
rs group                               RS1.L   ['RS1']
spirax-sarco engineering               SPX.L   ['SPX']
ashtead group                          AHT.L   ['AHT']
coca-cola hbc                          CCH.L   ['CCH']
rentokil initial                       RTO.L   ['RTO']
british american tobacco               BATS.L  ['BATS']
antofagasta                            ANTO.L  ['ANTO']
bae systems                            BA.L    ['BA.']
vodafone group public limited company  VOD.L   []
smiths group                           SMIN.L  ['SMIN']
schroders                              SDR.L   ['SDR', 'SDR.OLD', 'SDRC', 'INOV']
h

In [157]:
bad = []
for tk, query in companies:
    if tk in existing:
        continue
    ms = matches.get(query, [])
    if len(ms) == 0:
        bad.append((tk, query))
        print(tk, query)
    else:
        existing[tk] += 1
        data.extend({'ibkr': m[1].symbol, 'yahoo': tk} for m in ms)
for tk, q in bad:
    data.append({'ibkr': tk.split('.')[0], 'yahoo': tk})
    existing[tk] += 1
len(data)

VOD.L vodafone group public limited company
SMT.L scottish mortgage investment trust


293

In [158]:
save_yaml(data, path);

In [159]:
print(tabulate(bad))

-----  -------------------------------------
VOD.L  vodafone group public limited company
SMT.L  scottish mortgage investment trust
-----  -------------------------------------


In [170]:
tkquery = 'SMT'
out = ibe.search(tkquery)
options = [c for c in out if c.secType == 'STK']
print(tabulate([(c.symbol, c.currency, c.description) for c in reversed(options)]))

--------  ---  -----------------------------------------------
S4A.RTS2  EUR  SMT SCHARF AG - SUB RTS 210630
XM91      EUR  SMTC CORPORATION
GDIV      CAD  MANLFE SMT GLB DVD ETF PTFL
2642      JPY  SMT ETF CARBON EFFCNT IDX JP
00912     TWD  CTBC TIP CUS TW SMT FA50 ETF
S4AA      EUR  SMT SCHARF AG
SMTG.TEN  EUR  SUMMIT PROPERTIES LTD - TENDER INTERIM 16/03/20
SMTA.ESC  USD  SPIRIT MTA REIT - ESCROW
SMTC      MXN  SEMTECH CORP
SMTU      EUR  AMUNDI US SMRT ALOC SCI BETA
SMTSF     USD  SIERRA METALS INC
SMTH.P    CAD  SMITHE RESOURCES CORP
SMTK      USD  SMARTKEM INC
SMTTF     USD  SUN METALS CORP
SMTX      USD  SMTC CORPORATION
SMTC      USD  LYXOR SMART OVERNIGHT RETURN
SMTI      USD  SANARA MEDTECH INC
SMTG      EUR  SUMMIT PROPERTIES LTD
SMTC      USD  SEMTECH CORP
SMTO3     BRL  SAO MARTINHO SA
SMTOY     USD  SUMITOMO ELEC INDS-UNSP ADR
SMT       CAD  SIERRA METALS INC
SMT       ILS  SUMMIT REAL ESTATE HOLDING
SMT       GBP  SCOTTISH MORTGAGE INV TR PLC
--------  ---  ------------

In [171]:
ibtk = tkquery.upper()
info = xmltodict.parse(ibe.snapshot(ibtk))
# company_name = info['ReportSnapshot']['CoIDs']['CoID'][1]['#text']
industries = [e['#text'] for e in info['ReportSnapshot']['peerInfo']['IndustryInfo']['Industry']]
print('\n'.join(industries))
print()
summary = info['ReportSnapshot']['TextInfo']['Text'][0]['#text']
print(summary)

Diversified Mining
Gold Ore and Silver Ore Mining
Copper, Nickel, Lead, and Zinc Mining
Gold Ores
Copper Ores

Sierra Metals Inc. is a Canada-based diversified mining company. The Company is focused on Green Metal exposure, including copper, zinc and lead production with precious metals in Peru and Mexico. It owns and operates three mines in commercial production: the Yauricocha Mine in Peru, the Bolivar and Cusi mines in Mexico. The Yauricocha, is an underground mine located in western central Peru in the Yauyos province, 12 kilometers (km) west of the Continental Divide. Yauricocha Mine covers an area of 18,778 hectares (ha) that straddle a 20 km strike length of the prolific Yauricocha fault. The Bolivar Mine consists of approximately 12 mining concessions and is a contiguous portion of approximately 15,217 ha Bolivar property land package within the municipality of Urique, in the Piedras Verdes mining district of Chihuahua State, Mexico. The Cusi Mine is located in Chihuahua State,

In [101]:
ibe = IB_Extractor(True)

In [102]:
ibe.ib.disconnect()